# NIPS 2017 Figure 5

In [ ]:
%run -i NIPS2017.ipynb

# FIGURE and GRID
FIG_HEIGHT_MM = 40
FIG_WIDTH_MM = FIG_WIDTH_MM
FIG_N_ROWS = 1
ROW_1_NCOLS = 3
ROW_1_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_1_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_3_MM = FIG_WIDTH_MM / ROW_1_NCOLS

# PATHS
PANEL_A = PATH_DROPBOX_FIGS + 'fig5_a.svg'
PANEL_B = PATH_DROPBOX_FIGS + 'fig5_b.svg'
PANEL_C = PATH_DROPBOX_FIGS + 'fig5_c.svg'

## Row 1

Data for panels A

In [ ]:
# from lfmods.hh import HHSimulator

# sim = HHSimulator(seed=100,pilot_samples=0.,cached_sims=False, cached_pilot=False, step_current=False)

# # ground-truth parameters
# true_params = sim.true_params
# n_params = sim.n_params

# init = sim.init
# t = sim.t
# dt = sim.dt
# duration = np.max(t)
# I = sim.I_obs

# # prior over parameters
# sim_prior = sim.sim_prior
# sim_prior()
# prior_min = sim.prior_min
# prior_max = sim.prior_max

# # if parameters in transformed space, inverse transform function
# param_invtransform = sim.param_invtransform
# bm = sim.bm

# # simulate trace with spikes
# fact = 1.
# new_prior_min = fact*prior_min
# new_prior_max = fact*prior_maxx
# np.random.seed(123)
# z = np.random.rand(n_params)
# params = param_invtransform( np.multiply(new_prior_max - new_prior_min,z) + new_prior_min )
# hh = bm.HH(init, params.reshape(1,-1),seed=230)
# states_spikes = hh.sim_time(dt, t, I)[:,0]

Load results

In [ ]:
prefix = 'shrink_nips_v1_60ms'  # ours
_, _, _, _, posteriors, _ = io.load_prefix(dirs_dropbox['dir_nets_fig5'], prefix, no_net=True)
pdf60 = io.last(posteriors)

prefix = 'shrink_nips_v1_120ms'  # ours
_, _, _, _, posteriors, _ = io.load_prefix(dirs_dropbox['dir_nets_fig5'], prefix, no_net=True)
pdf120 = io.last(posteriors)

prefix = 'shrink_nips_v1_240ms'  # ours
_, _, _, _, posteriors, sims = io.load_prefix(dirs_dropbox['dir_nets_fig5'], prefix, no_net=True)
sim = io.last(sims)
obs_stats = sim.obs
y_obs = sim.obs_trace.reshape(-1,1)
gt = sim.true_params
labels_params = sim.labels_params
param_invtransform = sim.param_invtransform
param_transform = sim.param_transform
gt_transf = param_transform(gt)
n_params = sim.n_params
prior_min = sim.prior_min
prior_max = sim.prior_max
prior_lims = np.concatenate((prior_min.reshape(-1,1),prior_max.reshape(-1,1)),axis=1)
t = sim.t
I = sim.I_obs
A_soma = sim.A_soma
dt = sim.dt
duration = np.max(t)
pdf240 = io.last(posteriors)
bm = sim.bm

### Panel A

In [ ]:
fig_inches = (SCALE_IN*mm2inches(0.9*ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.95*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    ax = plt.subplot(gs[0])
    plt.plot(t, y_obs, color = COL['GT'], lw=2, label='')
    plt.ylabel('voltage (mV)')
    plt.legend(bbox_to_anchor=(1.15, 1), loc='upper right')

    ax.set_xticks([])
    ax.set_yticks([-80, -20, 40])

    ax = plt.subplot(gs[1])
    plt.plot(t,I*A_soma*1e3,'r', lw=2)
    plt.xlabel('time (ms)')
    plt.ylabel('input (nA)')

    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I*A_soma*1e3)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.2f'))
    
    plt.savefig(PANEL_A, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_A)

panel = st.fromfile(PANEL_A)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel B

Will be done in illustrator/omnigraffle

In [ ]:
try:
    svg(PANEL_B)
except:
    pass

print('width : {:g}mm'.format(ROW_1_WIDTH_COL_1_MM))
print('height : {:g}mm'.format(ROW_1_HEIGHT_MM))

### Panel C

Posterior view, marginals only

In [ ]:
fig_inches = (SCALE_IN*mm2inches(ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.95*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC, rc={'text.usetex': False}):

    cols = 4
    rows = 3
    
    num_samp = 3
    num_colors = num_samp+1
    cm1 = mpl.cm.Blues
    col1 = [cm1(1.*i/num_colors) for i in range(num_colors)]

    # load in a simulator from shrink_nips_v1_60ms or _120ms or 240ms, they do share the following variables:
    labels_params = LABELS_HH
    lims = np.array((sim.prior_min, sim.prior_max)).T
    gt = np.log(sim.true_params)
    resolution = 500
    ticks = False

    figsize = (4, 4)
    fig, ax = plt.subplots(rows, cols, facecolor='white', figsize=figsize)
    ax = ax.reshape(rows, cols)

    plt.hold(True)

    r = 0
    c = -1
    for i in range(n_params):
        xx = np.linspace(lims[i, 0], lims[i, 1], resolution)

        c += 1
        if c > cols-1:
            c = 0
            r += 1

        pp = pdf60.eval(xx, ii=[i], log=False)
        ax[r, c].plot(xx, pp, color=col1[1], alpha=0.9,label='60')  # need better colors, e.g. shades of blue light to dark

        pp = pdf120.eval(xx, ii=[i], log=False)
        ax[r, c].plot(xx, pp, color=col1[2], alpha=0.9,label='120')  # need better colors, e.g. shades of blue light to dark

        pp = pdf240.eval(xx, ii=[i], log=False)
        ax[r, c].plot(xx, pp, color=col1[3], alpha=0.9,label='240')  # need better colors, e.g. shades of blue light to dark

        ax[r, c].set_xlim(lims[i])
        ax[r, c].set_ylim([0, ax[r, c].get_ylim()[1]])      

        if gt is not None: ax[r, c].vlines(gt[i], 0, ax[r, c].get_ylim()[1], color='r')

        if ticks:
            ax[r, c].get_yaxis().set_tick_params(which='both', direction='out')
            ax[r, c].get_xaxis().set_tick_params(which='both', direction='out')
            ax[r, c].set_xticks(np.linspace(lims[i, 0], lims[j, 1],2))
            ax[r, c].set_yticks(np.linspace(min(pp),max(pp),2))
            ax[r, c].xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
            ax[r, c].yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
        else:
            ax[r, c].get_xaxis().set_ticks([])
            ax[r, c].get_yaxis().set_ticks([])

        if labels_params is not None:
            ax[r, c].set_xlabel(labels_params[i])
        else:
            ax[r, c].set_xlabel([])

        x0,x1 = ax[r, c].get_xlim()
        y0,y1 = ax[r, c].get_ylim()
        ax[r, c].set_aspect((x1-x0)/(y1-y0))
    
        if i == 3:
            ax[r, c].legend(bbox_to_anchor=(2.7, 1.2), loc='upper right')
    
    plt.savefig(PANEL_C, facecolor=plt.gcf().get_facecolor(), transparent=True)
    plt.close()
    
svg(PANEL_C)

panel = st.fromfile(PANEL_C)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

## Compose figure

In [ ]:
fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)

yoffset = -1.2
xoffset = -1.
fig = add_svg(fig, PANEL_A, 0 + xoffset, 0 + yoffset + 1)
fig = add_svg(fig, PANEL_B, ROW_1_WIDTH_COL_1_MM + xoffset, 0 + 1.3 + yoffset, scale=0.5)
fig = add_svg(fig, PANEL_C, ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM + xoffset + 3., 0 + yoffset + 1)

yoffset   = 2.3
fig = add_label(fig, 'A', 0, yoffset)
fig = add_label(fig, 'B', ROW_1_WIDTH_COL_1_MM, yoffset)
fig = add_label(fig, 'C', ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM, yoffset)

if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)

PATH_SVG = PATH_DROPBOX_FIGS + 'fig5.svg'
fig.save(PATH_SVG)

svg(PATH_SVG)

!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig5.pdf $PATH_SVG